# Pipeline 2
This pipeline will be very similar with Pipeline 1, but we will use the Title from peoples names as features

## Import the Data

In [21]:
import pandas as pd

trainData = pd.read_csv("../data/raw/train.csv")
trainData.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [22]:
testData = pd.read_csv("../data/raw/test.csv")
testData.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


## Prepare the data

### Create the Title column

In [23]:
datasets = [trainData, testData]

for dataset in datasets:
    dataset["Title"] = [name.split(",")[1].split(".")[0].lstrip() for name in dataset["Name"]]

trainData.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,Mr
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Mrs
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Miss
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Mrs
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,Mr


### Get titles with less than 10 occurrences

In [24]:
trainData["Title"].value_counts()

Title
Mr              517
Miss            182
Mrs             125
Master           40
Dr                7
Rev               6
Mlle              2
Major             2
Col               2
the Countess      1
Capt              1
Ms                1
Sir               1
Lady              1
Mme               1
Don               1
Jonkheer          1
Name: count, dtype: int64

In [25]:
titlesToReplace = trainData["Title"].value_counts().gt(10)
titlesToReplace = titlesToReplace.index[titlesToReplace == False].values
print(titlesToReplace)

['Dr' 'Rev' 'Mlle' 'Major' 'Col' 'the Countess' 'Capt' 'Ms' 'Sir' 'Lady'
 'Mme' 'Don' 'Jonkheer']


### Replace titles with less than 10 occurrences with either Mr or Miss

In [26]:
for title in titlesToReplace:
    for dataset in datasets:
        dataset.loc[(dataset["Title"] == title) & (dataset["Sex"] == "male"), "Title"] = "Mr"
        dataset.loc[(dataset["Title"] == title) & (dataset["Sex"] == "female"), "Title"] = "Miss"
        
trainData["Title"].value_counts()

Title
Mr        537
Miss      189
Mrs       125
Master     40
Name: count, dtype: int64

### Drop the columns that we will not use

In [27]:
for dataset in datasets:
    dataset.drop("PassengerId", axis=1, inplace=True)
    dataset.drop("Name", axis=1, inplace=True)
    dataset.drop("Ticket", axis=1, inplace=True)
    dataset.drop("Cabin", axis=1, inplace=True)

In [28]:
trainData.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title
0,0,3,male,22.0,1,0,7.2500,S,Mr
1,1,1,female,38.0,1,0,71.2833,C,Mrs
2,1,3,female,26.0,0,0,7.9250,S,Miss
3,1,1,female,35.0,1,0,53.1000,S,Mrs
4,0,3,male,35.0,0,0,8.0500,S,Mr


### Handle missing values

In [29]:
missingData = []

for dataset in datasets:
    totalMissing = dataset.isnull().sum().sort_values(ascending=False)
    percent = dataset.isnull().sum() / dataset.isnull().count() * 100
    percent = round(percent, 1).sort_values(ascending=False)
    
    missingData.append(pd.concat([totalMissing, percent], axis=1, keys=["Total", "%"]))
    
pd.concat(missingData, axis=1, keys=["Training Data", "Testing Data"])

Training Data       Testing Data      
                 Total     %        Total     %
Age                177  19.9         86.0  20.6
Embarked             2   0.2          0.0   0.0
Survived             0   0.0          NaN   NaN
Pclass               0   0.0          0.0   0.0
Sex                  0   0.0          0.0   0.0
SibSp                0   0.0          0.0   0.0
Parch                0   0.0          0.0   0.0
Fare                 0   0.0          1.0   0.2
Title                0   0.0          0.0   0.0

In [30]:
for dataset in datasets:
    meanAge = dataset["Age"].mean()
    mostFrequentEmbarked = dataset["Embarked"].mode().item()
    meanFare = dataset["Fare"].mean()
    
    dataset["Age"].fillna(meanAge, inplace=True)
    dataset["Embarked"].fillna(mostFrequentEmbarked, inplace=True)
    dataset["Fare"].fillna(meanFare, inplace=True)

In [31]:
missingData = []

for dataset in datasets:
    totalMissing = dataset.isnull().sum().sort_values(ascending=False)
    percent = dataset.isnull().sum() / dataset.isnull().count() * 100
    percent = round(percent, 1).sort_values(ascending=False)
    
    missingData.append(pd.concat([totalMissing, percent], axis=1, keys=["Total", "%"]))
    
pd.concat(missingData, axis=1, keys=["Training Data", "Testing Data"])

Training Data      Testing Data     
                 Total    %        Total    %
Survived             0  0.0          NaN  NaN
Pclass               0  0.0          0.0  0.0
Sex                  0  0.0          0.0  0.0
Age                  0  0.0          0.0  0.0
SibSp                0  0.0          0.0  0.0
Parch                0  0.0          0.0  0.0
Fare                 0  0.0          0.0  0.0
Embarked             0  0.0          0.0  0.0
Title                0  0.0          0.0  0.0

### Normalize values

In [32]:
for dataset in datasets:
    pclassMax = dataset["Pclass"].max()
    ageMax = dataset["Age"].max()
    sibSpMax = dataset["SibSp"].max()
    parchMax = dataset["Parch"].max()
    fareMax = dataset["Fare"].max()

    dataset["Pclass"] = dataset["Pclass"] / pclassMax
    dataset["Age"] = dataset["Age"] / ageMax
    dataset["SibSp"] = dataset["SibSp"] / sibSpMax
    dataset["Parch"] = dataset["Parch"] / parchMax
    dataset["Fare"] = dataset["Fare"] / fareMax

In [33]:
trainData.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title
0,0,1.000000,male,0.2750,0.125,0.0,0.014151,S,Mr
1,1,0.333333,female,0.4750,0.125,0.0,0.139136,C,Mrs
2,1,1.000000,female,0.3250,0.000,0.0,0.015469,S,Miss
3,1,0.333333,female,0.4375,0.125,0.0,0.103644,S,Mrs
4,0,1.000000,male,0.4375,0.000,0.0,0.015713,S,Mr


### One-Hot Encode categorical features

In [34]:
trainData = pd.get_dummies(trainData, columns=["Embarked", "Sex", "Title"], dtype=int)
testData = pd.get_dummies(testData, columns=["Embarked", "Sex", "Title"], dtype=int)

In [35]:
trainData.head()

,Survived,Pclass,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S,Sex_female,Sex_male,Title_Master,Title_Miss,Title_Mr,Title_Mrs
0,0,1.000000,0.2750,0.125,0.0,0.014151,0,0,1,0,1,0,0,1,0
1,1,0.333333,0.4750,0.125,0.0,0.139136,1,0,0,1,0,0,0,0,1
2,1,1.000000,0.3250,0.000,0.0,0.015469,0,0,1,1,0,0,1,0,0
3,1,0.333333,0.4375,0.125,0.0,0.103644,0,0,1,1,0,0,0,0,1
4,0,1.000000,0.4375,0.000,0.0,0.015713,0,0,1,0,1,0,0,1,0


## Separate Features and Labels

In [36]:
trainFeatures = trainData.drop("Survived", axis=1)
trainLabels = trainData["Survived"]

## Save new Training and Testing Datasets to csv

In [37]:
trainFeatures.to_csv("../data/preprocessed/pipeline2/trainFeatures.csv", index=False)
trainLabels.to_csv("../data/preprocessed/pipeline2/trainLabels.csv", index=False)
testData.to_csv("../data/preprocessed/pipeline2/test.csv", index=False)